## 네이버 영화리뷰 감성분석
[e9t/nsmc: Naver sentiment movie corpus](https://github.com/e9t/nsmc)

In [0]:
# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 
%config InlineBackend.figure_formats = ['retina']

## 데이터 로드하기

In [0]:
train = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep="\t")
train.shape

In [0]:
test = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep="\t")
test.shape

In [0]:
train.head()

In [0]:
test.head()

In [0]:
train.info()

### 중복 데이터 제거

In [0]:
train = train.drop_duplicates(["document"])
train.shape

In [0]:
test = test.drop_duplicates(["document"])
test.shape

### 결측치 제거

In [0]:
train = train[train["document"].notnull()]
train.shape

In [0]:
test = test[test["document"].notnull()]
test.shape

### 빠른 학습을 위해 일부만 샘플링

In [0]:
train = train.sample(10000, random_state=1)
train.shape

In [0]:
test = test.sample(5000, random_state=1)
test.shape

### 결측치 확인

In [0]:
train["document"].isnull().sum()

In [0]:
test["document"].isnull().sum()

## 정답값 보기

In [0]:
train["label"].value_counts()

In [0]:
train["label"].value_counts(normalize=True)

In [0]:
sns.countplot(data=train, x="label")

## 문서 분석하기

In [0]:
train["document"].nunique()

### 중복값 보기

In [0]:
train["document"].nunique()

In [0]:
train.loc[train["document"].duplicated(), "document"].value_counts()

### 단어 수 세기

In [0]:
train["len"] = train["document"].apply(lambda x : len(x))
train["word_count"] = train["document"].apply(lambda x : len(x.split()))
train["unique_word_count"] = train["document"].apply(lambda x : len(set(x.split())))

In [0]:
train[["len", "word_count", "unique_word_count"]].describe()

In [0]:
sns.distplot(train["len"], hist=False, label="단어 길이")
sns.distplot(train["word_count"], hist=False, label="단어 수")
sns.distplot(train["unique_word_count"], hist=False, label="중복 제거 단어 수")

### 용어 정리

- 배치(batch): 모델 학습에 한 번에 입력할 데이터셋

- 에폭(epoch): 모델 학습시 전체 데이터를 학습한 횟 수

- 스텝(step): (모델 학습의 경우) 하나의 배치를 학습한 횟 수

- 토큰(token): (여기서는) 문장 또는 문단의 기본 구성 단위 (예를 들어 단위, 형태소)

- 토크나이징(tokenizing): 문단 또는 문장 문자열을 하나의 토큰 단위로 쪼개는 작업

- 인덱싱(indexing): 토큰 문자열을 숫자(인덱스)로 변환하여 표현하는 작업

- 사전(vocabulary): 토큰 문자열과 인덱스의 관계를 정의해둔 셋

- n-그램(n-gram): 입력한 문자열을 N개의 기준 단위로 절단하는 방법

In [0]:
import re

def preprocessing(text):
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub(' +', ' ', text)
    # 중복으로 생성된 ㅋ 값을 제거합니다.
    text = re.sub('ㅋ+', 'ㅋ', text)
    return text

In [0]:
# 아래의 결과가  "재미있었어요ㅋ" 가 나오도록 위 preprocessing 을 고쳐보세요.

preprocessing("재미있었어요ㅋㅋㅋㅋㅋ")

In [0]:
# preprocessing 함수를 적용합니다.
train["document"] = train["document"].apply(preprocessing)
test["document"] = test["document"].apply(preprocessing)

### 형태소 분석기 사용하기

In [0]:
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# cd Mecab-ko-for-Google-Colab/
# !bash install_mecab-ko_on_colab190912.sh

In [0]:
# 한국어 형태소 분석기 설치
!pip install konlpy

### 불용어 제거

In [0]:
from konlpy.tag import Okt
okt=Okt()

# 불용어 제거
def remove_stopwords(text):
    # okt 객체를 활용해서 형태소 단위로 나눈다.
    text = okt.morphs(text, stem=True)
    stops = ['안녕', '안녕하세요', '있습니다', '하는', 
             '그', '및', '제', '할', '하고', '더', '대한', 
             '한', '그리고', '월', '저는', '없는', '입니다', 
             '등', '일', '많은', '이런', '것은', '왜','같은', 
             '같습니다', '없습니다', '위해', '한다']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [0]:
from tqdm import tqdm
tqdm.pandas()
# remove_stopwords 함수를 적용합니다.
train["document"] = train["document"].progress_apply(remove_stopwords)
test["document"] = test["document"].progress_apply(remove_stopwords)

### 워드클라우드로 빈도수 표현하기
[amueller/word_cloud: A little word cloud generator in Python](https://github.com/amueller/word_cloud)

* 별도의 설치가 필요합니다. 
* 다음 명령어로 설치가 가능합니다. conda prompt 혹은 터미널을 열어 설치해 주세요.

* conda 사용시 : `conda install -c conda-forge wordcloud`
* pip 사용시 : `pip install wordcloud`

In [0]:
# 공식문서의 튜토리얼을 보고 wordcloud를 그리는 함수를 만들어 봅니다.
# 이때 폰트 설정시 폰트명이 아닌 폰트의 설치 경로를 입력해 주셔야 합니다.
# 윈도우 :  r"C:\Windows\Fonts\Malgun Gothic.ttf" 해당 경로에 폰트가 있는지 확인을 해주세요.
# 맥 : r"/Library/Fonts/AppleGothic.ttf"
# 나눔고딕 등의 폰트를 설치했다면 : '/Library/Fonts/NanumBarunGothic.ttf'

from wordcloud import WordCloud

def wordcloud(data, width=1200, height=500):
    word_draw = WordCloud(
        font_path=fontpath,
        width=width, height=height,
        stopwords=["영화"], 
        background_color="white",
        random_state=42
    )
    word_draw.generate(data)

    plt.figure(figsize=(15, 7))
    plt.imshow(word_draw)
    plt.axis("off")
    plt.show()

In [0]:
wordcloud(" ".join(train["document"]))

In [0]:
from keras.preprocessing.text import Tokenizer
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(train["document"])

X_train = tokenizer.texts_to_sequences(train["document"])
X_test = tokenizer.texts_to_sequences(test["document"])

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 128 # 전체 데이터의 길이를 128

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [0]:
y_train = train["label"]
X_train.shape, len(y_train), X_test.shape

In [0]:
max_words

### 신경망 구성

<img src="https://cs231n.github.io/assets/nn1/neural_net2.jpeg" width=500>

### Layer 만들기

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(0)

# tf.keras.layers.Embedding(max_words, max_len) 으로 임베딩을 하고 
# 입력 : 기본 레이어 16개의 노드를 갖는 Dense 레이어를 구성하고 활성화 함수로 relu를 사용하는 레이어 2개를 작성합니다.
# 출력 : sigmoid 5개로 출력하는 모델을 작성합니다. evaluate 를 해보고 softmax 로 변경 후 값의 변화를 확인합니다.
# 모델작성 후 summary로 요약을 봅니다.

model = Sequential()
model.add(#워드 임베딩)
model.add(# 16개 노드 Dense 레이어, 활성화 함수 relu )
model.add(# 16개 노드 Dense 레이어, 활성화 함수 relu )
model.add(# 5개의 출력을 갖는 Dense 레이어, 활성화 함수 softmax )
model.summary()

### model.compile

In [0]:
# model을 compile 합니다.
# optimizer 를 설정합니다. 기본은 adam 부터 시작해 봅니다.
# 측정은 accuracy 로 합니다.
model.compile(optimizer='_____',
              loss='sparse_categorical_crossentropy',
              metrics=['______'])

In [0]:
# Early Stopping patience=3는 val_accuracy가 3번이상 증가하지 않으면 중지
# 아래 빈칸에 patience 를 채워주세요.
callbacks = [EarlyStopping(monitor='val_accuracy', _______=3)] 

### 학습

In [0]:
# 학습
# callbacks설정을 추가해 봅니다.
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split = 0.2, 
          callbacks = callbacks)

### 예측

In [0]:
y_predict = model.predict(X_test)
y_predict

In [0]:
y_test = test["label"]
y_test.shape

In [0]:
X_test.shape, y_test.shape

### 평가

In [0]:
results = model.evaluate(X_test,  y_test, verbose=2)
results